<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 3. Обучение с учителем. Методы классификации
## <center>Практика. Дерево решений в задаче предсказания выживания пассажиров "Титаника". Решение

**Заполните код в клетках и выберите ответы в [веб-форме](https://goo.gl/TBwVhQ).**

**<a href="https://www.kaggle.com/c/titanic">Соревнование</a> Kaggle "Titanic: Machine Learning from Disaster".**

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

**Функция для формирования csv-файла посылки на Kaggle:**

In [37]:
def write_to_submission_file(
    predicted_labels,
    out_file,
    train_num=891,
    target="Survived",
    index_label="PassengerId",
):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(
        predicted_labels,
        index=np.arange(train_num + 1, train_num + 1 + predicted_labels.shape[0]),
        columns=[target],
    )
    predicted_df.to_csv(out_file, index_label=index_label)

**Считываем обучающую и тестовую выборки.**

In [44]:
df = pd.read_csv("../../data/titanic_train.csv")

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Embarked     891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [60]:
df.drop('Ticket', axis=1, inplace=True)

In [48]:
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Embarked"].fillna("S", inplace=True)
df["Fare"].fillna(df["Fare"].median(), inplace=True)

In [77]:
X = df

In [85]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int32

In [79]:
X = pd.concat(
    [
        X,
        pd.get_dummies(X["Pclass"], prefix="PClass"),
        pd.get_dummies(X["Sex"], prefix="Sex"),
        pd.get_dummies(X["SibSp"], prefix="SibSp"),
        pd.get_dummies(X["Parch"], prefix="Parch"),
        pd.get_dummies(X["Embarked"], prefix="Embarked"),
    ],
    axis=1,
)

In [83]:
X.drop(['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked'], axis=1, inplace=True)

In [84]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Age          891 non-null    float64
 2   Fare         891 non-null    float64
 3   PClass_1     891 non-null    uint8  
 4   PClass_2     891 non-null    uint8  
 5   PClass_3     891 non-null    uint8  
 6   Sex_female   891 non-null    uint8  
 7   Sex_male     891 non-null    uint8  
 8   SibSp_0      891 non-null    uint8  
 9   SibSp_1      891 non-null    uint8  
 10  SibSp_2      891 non-null    uint8  
 11  SibSp_3      891 non-null    uint8  
 12  SibSp_4      891 non-null    uint8  
 13  SibSp_5      891 non-null    uint8  
 14  SibSp_8      891 non-null    uint8  
 15  Parch_0      891 non-null    uint8  
 16  Parch_1      891 non-null    uint8  
 17  Parch_2      891 non-null    uint8  
 18  Parch_3      891 non-null    uint8  
 19  Parch_4 

In [101]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,test_size=0.3, random_state=17)

In [105]:
X_train.shape, X_test.shape

((623, 25), (268, 25))

## 1. Дерево решений без настройки параметров 

**Обучите на имеющейся выборке дерево решений (`DecisionTreeClassifier`) максимальной глубины 2. Используйте параметр `random_state=17` для воспроизводимости результатов.**

In [91]:
first_tree = DecisionTreeClassifier(random_state=17)

In [106]:
first_tree.fit(X_train, y_train);

In [107]:
np.mean(sklearn.model_selection.cross_val_score(first_tree, train_df, y, cv=5))

0.7755570899504112

In [108]:
1 - np.mean(y)

0.6161616161616161

In [110]:
tree_test_pred = first_tree.predict(X_test)
accuracy_score(y_test, tree_test_pred)

0.7164179104477612

**Сделайте с помощью полученной модели прогноз для тестовой выборки **

**Сформируйте файл посылки и отправьте на Kaggle**

In [128]:
write_to_submission_file(tree_test_pred, 'file1.csv')

<font color='red'>Вопрос 1. </font> Каков результат первой посылки (дерево решений без настройки параметров) в публичном рейтинге соревнования Titanic?
- 0.746
- 0.756
- 0.766
- 0.776

**Отобразите дерево с помощью `export_graphviz` и `dot`.**

In [22]:
# Ваш код здесь

<font color='red'>Вопрос 2. </font> Сколько признаков задействуются при прогнозе деревом решений глубины 2?
- 2
- 3
- 4
- 5

## 2. Дерево решений с настройкой параметров 

**Обучите на имеющейся выборке дерево решений (`DecisionTreeClassifier`). Также укажите `random_state=17`. Максимальную глубину и минимальное число элементов в листе настройте на 5-кратной кросс-валидации с помощью `GridSearchCV`.**

In [121]:
from sklearn.model_selection import GridSearchCV
tree_params = {"max_depth": list(range(1, 5)), "min_samples_leaf": list(range(1, 5))} #, "max_features": [0.5, 0.7, 1]
tree_grid = GridSearchCV(first_tree, tree_params)
# Ваш код здесь

In [122]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 275 ms


GridSearchCV(estimator=DecisionTreeClassifier(random_state=17),
             param_grid={'max_depth': [1, 2, 3, 4],
                         'min_samples_leaf': [1, 2, 3, 4]})

In [124]:
tree_grid.best_params_, tree_grid.best_score_

({'max_depth': 3, 'min_samples_leaf': 4}, 0.8313806451612903)

<font color='red'>Вопрос 3. </font> Каковы лучшие параметры дерева, настроенные на кросс-валидации с помощью `GridSearchCV`?
- max_depth=2, min_samples_leaf=1
- max_depth=2, min_samples_leaf=4
- max_depth=3, min_samples_leaf=2
- max_depth=3, min_samples_leaf=3

<font color='red'>Вопрос 4. </font> Какой получилась средняя доля верных ответов на кросс-валидации для дерева решений с лучшим сочетанием гиперпараметров `max_depth` и `min_samples_leaf`?
- 0.77
- 0.79
- 0.81
- 0.83

**Сделайте с помощью полученной модели прогноз для тестовой выборки.**

In [126]:
accuracy_score(y_test, tree_grid.predict(X_test))

0.7611940298507462

**Сформируйте файл посылки и отправьте на Kaggle.**

In [25]:
# Ваш код здесь

<font color='red'>Вопрос 5. </font> Каков результат второй посылки (дерево решений с настройкой гиперпараметров) в публичном рейтинге соревнования Titanic?
- 0.7499
- 0.7599
- 0.7699
- 0.7799

## Ссылки:

 - <a href="https://www.kaggle.com/c/titanic">Соревнование</a> Kaggle "Titanic: Machine Learning from Disaster"
 - <a href="https://www.dataquest.io/mission/74/getting-started-with-kaggle/">Тьюториал</a> Dataquest по задаче Kaggle "Titanic: Machine Learning from Disaster"